In [1]:
#imports
import base64
import sys
import time
import pyvisa
from time import sleep
from toptica.lasersdk.client import Client, SerialConnection, UserLevel

In [2]:
#set connections to DLC smart and lock-in amplifier
lockin_address = 'GPIB::8::INSTR'
dlc_connection_port = 'COM3'
#lock-in sensitivity setting
sensitivity_setting = 15
#set frequency to measure at
set_frequency = 180 #GHz
time_constant = 8 #lock-in time constant setting, specifics found in SR830 manual
low_pass_filter = 0 #lock-in low pass filter slope setting, specifics found in SR830 manual

In [3]:
#set up connection with lock-in amplifier
rm = pyvisa.ResourceManager()
lockin = rm.open_resource(lockin_address)
idn = lockin.query('*IDN?')
print(f'Connected to: {idn}')

Connected to: Stanford_Research_Systems,SR830,s/n46458,ver1.07 



In [4]:
#set up connection with DLC smart and check user level
with Client(SerialConnection(dlc_connection_port)) as client:
    user_level = client.get('ul')
    dlc_smart = client.get('general:system-type')
    print(f'Connected to: {dlc_smart}')
    print(f"Current User Level: {user_level}")

Connected to: DLCsmartTHz
Current User Level: 2


In [5]:
#turn on lasers (check emission lights)
#if lasers do not power on, check the frontkey and user level
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('laser-operation:emission-global-enable', True)
    print(f'Lasers on: {client.get('laser-operation:emission-global-enable')}')
#wait 5 secs for lasers to adjust
time.sleep(5)

Lasers on: True


In [6]:
#set DLC Smart bias amplitude and offset to default
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('lockin:mod-out-amplitude', 0.80)
    client.set('lockin:mod-out-offset', -0.30)
#set DLC Smart bias frequency to 1000
    client.set('lockin:frequency', 1000)
    print(f"Current DLC Smart Bias Amplitude (V): {client.get('lockin:mod-out-amplitude')}")
    print(f"Current DLC Smart Bias Offset (V): {client.get('lockin:mod-out-offset')}")
    print(f"Current DLC Smart Bias Frequency (Hz): {client.get('lockin:frequency')}")
#wait 5 secs for DLC smart to adjust
time.sleep(5)

Current DLC Smart Bias Amplitude (V): 0.800018310546875
Current DLC Smart Bias Offset (V): -0.30002441704309607
Current DLC Smart Bias Frequency (Hz): 1000


In [7]:
#set DLC Smart frequency to value you wish to measure
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('frequency:frequency-set', set_frequency)
    print(f"Current DLC Smart Frequency (GHz): {client.get('frequency:frequency-act')}")

Current DLC Smart Frequency (GHz): 180.0027486262552


In [9]:
#set lock-in amplifier sensitivity
lockin.write(f'SENS {sensitivity_setting}')
lockin_sensitivity = lockin.query('SENS?')
print(f"Current Lock-in Sensitivity setting: {lockin_sensitivity}")
#set lock-in time constant setting
lockin.write(f'OFLT {time_constant}')
print(f'Current Time Constant: {lockin.query('OFLT?')}')
#set lock-in low pass filter slope
lockin.write(f'OFSL {low_pass_filter}')
print(f'Current Low Pass Filter: {lockin.query('OFSL?')}')

Current Lock-in Sensitivity setting: 15

Current Time Constant: 8

Current Low Pass Filter: 0

